In [7]:
import cv2
import pickle
import numpy as np

In [8]:
ESCAPE_KEY = 27
SPACE_KEY = 32
MARKER_SIZE = 0.05

In [9]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

with open("calibration_parameters.pickle", 'rb') as file:
    calib_params = pickle.load(file)

In [10]:
def track_aruco(frame):
    obj_points = np.array([[-MARKER_SIZE / 2, MARKER_SIZE / 2, 0],
                       [MARKER_SIZE / 2, MARKER_SIZE / 2, 0],
                       [MARKER_SIZE / 2, -MARKER_SIZE / 2, 0],
                       [-MARKER_SIZE / 2, -MARKER_SIZE / 2, 0]], dtype=np.float32)

    corners, ids, rejected_img_points = detector.detectMarkers(frame)
    cv2.aruco.drawDetectedMarkers(frame, corners, ids)
    for i in range(len(corners)):
        _, rvecs, tvecs = cv2.solvePnP(obj_points, corners[i], calib_params['mtx'], calib_params['dist'])
        cv2.drawFrameAxes(frame, calib_params['mtx'], calib_params['dist'], rvecs, tvecs, MARKER_SIZE * .5, 2)
        distance = np.linalg.norm(tvecs)
        image = cv2.putText(frame, str(distance), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA) 
        return (tvecs, rvecs)

In [11]:
def track(vid):
    home = None
    while True:
        ret, frame = vid.read()
        
        current_state = track_aruco(frame)
        key = cv2.waitKey(3)
        if key == ESCAPE_KEY:
            break
        if key == SPACE_KEY:
            home = current_state
        if home and current_state:
            vec_to_home = np.transpose(home[1] - current_state[1])[0]
            cv2.putText(frame, f"X-axis: {round(vec_to_home[0]*100)}cm", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA) 
            cv2.putText(frame, f"Y-axis: {round(vec_to_home[1]*100)}cm", (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA) 
            cv2.putText(frame, f"Z-axis: {round(vec_to_home[2]*100)}cm", (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA) 
        cv2.imshow('frame', frame)

        

    
    vid.release()
    cv2.destroyAllWindows()

In [12]:
# Get camera feed
vid = cv2.VideoCapture(0) 
track(vid)